## Slides For DSS Project - THis is PAin

## Using the Lakh MIDI Dataset Clean  from Kaggle:

https://www.kaggle.com/datasets/imsparsh/lakh-midi-clean/discussion?resource=download

    - This can be changed later to give you different results... stay tuned


### What is a genetic algoryhm and what are the steps involved? 
https://www.youtube.com/watch?v=uQj5UNhCPuo&t=451s 

- Genetic representation of the solution
- Function to generate new solutions
- Fitness function
- Selection Function
- Crossover Function
- Mutation Function

### What are the steps in a data science project?




#### Importing required modules

In [1]:
import os
import random
import music21
from music21 import stream, duration, midi

#### Music 21
https://github.com/cuthbertLab/music21 (pip install music21)

#### Defining a function to extract the melody and chord progression from the midi files

In [2]:
def extract_features(filename):
    # Load the MIDI file into a music21 Stream object
    midi = music21.converter.parse(filename)
    
    # Extract the melody from the MIDI file
    melody = midi.parts[0].flat.notes
    
    # Extract the chord progression from the MIDI file
    chords = []
    for measure in midi.parts[1].measures(1, len(midi.parts[1])):
        chord = measure.chordify()
        chords.extend(chord.flat.getElementsByClass(music21.chord.Chord))
    
    return melody, chords

#### Get features from midi dataset

In [3]:
midi_dir = "music_gen/data/midi_dataset/ABBA"

old_music = []
for file in os.listdir(midi_dir):
    if file.endswith(".mid") or file.endswith(".midi"):
    filename = os.path.join(midi_dir, file)
    midi = music21.converter.parse(filename)
    if len(midi.parts) == 2:
        melody, chords = extract_features(filename)
        sequence = [note.nameWithOctave for note in melody] + [chord.normalOrder for chord in chords]
        old_music.append(sequence)
    else:
        print(f"{filename} does not have the expected number of parts.")

#### Define Fitness Function(s)
- Different functions can be used to generate different outcomes

In [4]:
def evaluate_fitness(sequence):
    # Compute the number of matching notes and chords between the generated sequence and the original music
    matches = sum(1 for note in sequence if note in old_music)
    
    # Compute the total length of the generated sequence and the original music
    total_length = len(sequence) + sum(len(s) for s in old_music)
    
    # Check if total_length is greater than zero to avoid zero division error
    if total_length > 0:
        # Compute the fitness score as the ratio of matching notes and chords to the total length
        fitness_score = matches / total_length
    else:
        # If total_length is zero, set the fitness score to zero
        fitness_score = 0
        
    return fitness_score


def calculate_fitness(population):
    """
    Calculate the fitness score for each individual in the population
    """
    fitness_scores = []
    for individual in population:
        # Convert the individual to a music sequence
        sequence = individual_to_sequence(individual)
        
        # Evaluate the fitness of the sequence using the evaluate_fitness() function
        fitness_score = evaluate_fitness(sequence)
        
        # Append the fitness score to the list of fitness scores
        fitness_scores.append(fitness_score)
        
    return fitness_scores

### Starting Genetic Algorythm

In [5]:
#### Function and parameters to generate random music

In [6]:
POPULATION_SIZE = 100
MUTATION_RATE = 0.1
GENERATIONS = 50
length = len(old_music)


def generate_sequence(length):
    sequence = [random.choice(old_music) for j in range(length)]
    return sequence

def generate_population(length):
    population = []
    for i in range(POPULATION_SIZE):
        sequence = generate_sequence(length)
        population.append(sequence)
        
    population = [generate_sequence() for i in range(POPULATION_SIZE)]
    while not population:  # Check if population is empty
        population = [generate_sequence() for i in range(POPULATION_SIZE)]

    return population

#### Selection function

In [7]:
def select_top_performers(population, fitness_fn):
    # Sort the population by fitness score
    sorted_population = sorted(population, key=fitness_fn, reverse=True)
    # Select the top performers to be parents for the next generation
    top_performers = sorted_population[:int(POPULATION_SIZE/2)]
    return top_performers

#### Crossover function

In [8]:
# Define a function to perform crossover between parents to create offspring
def crossover(parent1, parent2):
    # Implement crossover logic here
    # Example crossover function that randomly selects notes from each parent to create an offspring:
    offspring = []
    for i in range(len(parent1)):
        if random.random() < 0.5:
            offspring.append(parent1[i])
        else:
            offspring.append(parent2[i])
    return offspring

#### Mutation Function

In [9]:
def mutate(sequence):
    # Implement mutation logic here
    # Example mutation function that randomly changes a note in the sequence:
    if random.random() < MUTATION_RATE:
        index = random.randint(0, len(sequence)-1)
        sequence[index] = random.choice(old_music)
    return sequence

#### Implementation

In [11]:
population = generate_population(length=len(old_music))

for generation in range(GENERATIONS):
    # Evaluate the fitness of each sequence in the population
    fitness_scores = [evaluate_fitness(sequence) for sequence in population]
    
    # Select the top performers to be parents for the next generation
    selected_top_performers = select_top_performers(population, evaluate_fitness)
    
    # Create the next generation by performing crossover and mutation on the parents
    next_generation = selected_top_performers[:]

    
    while len(next_generation) < POPULATION_SIZE:
        parent1 = random.choice(selected_top_performers)
        parent2 = random.choice(selected_top_performers)
        offspring = crossover(parent1, parent2)
        offspring = mutate(offspring)
        next_generation.append(offspring)
    
    # Replace the old population with the new one for the next generation
    population = next_generation

# Select the best sequence from the final generation
best_sequence = max(population, key=evaluate_fitness)

TypeError: generate_sequence() missing 1 required positional argument: 'length'

#### Playing Output

In [ ]:
def play_sequence(sequence):
    # Create a new music stream
    s = stream.Stream()

    # Add each note or chord in the sequence to the stream
    for note_or_chord in sequence:
        # Create a new note or chord object from the string representation
        n = music21.note.Note(note_or_chord)

        # Set the duration of the note or chord to a quarter note
        n.duration = duration.Duration(0.25)

        # Add the note or chord to the stream
        s.append(n)

    # Save the stream as a MIDI file
    s.write('midi', fp='generated_sequence.mid')

    # Play the MIDI file using the default system player
    midi.realtime.StreamPlayer(s).play()

In [ ]:
import os
import random
import music21
from music21 import stream, duration, midi

# Defining a function to extract the melody and chord progression from the midi files
def extract_features(filename):
    # Load the MIDI file into a music21 Stream object
    midi = music21.converter.parse(filename)
    
    # Extract the melody from the MIDI file
    melody = midi.parts[0].flat.notes
    
    # Extract the chord progression from the MIDI file
    chords = []
    for measure in midi.parts[1].measures(1, len(midi.parts[1])):
        chord = measure.chordify()
        chords.extend(chord.flat.getElementsByClass(music21.chord.Chord))
    
    return melody, chords

# Get features from midi dataset
midi_dir = "music_gen/data/midi_dataset"

old_music = []
for file in os.listdir(midi_dir):
    if file.endswith(".mid") or file.endswith(".midi"):
        filename = os.path.join(midi_dir, file)
        melody, chords = extract_features(filename)
        sequence = [note.nameWithOctave for note in melody] + [chord.normalOrder for chord in chords]
        old_music.append(sequence)

# Define Fitness Function(s)
def evaluate_fitness(sequence):
    # Compute the number of matching notes and chords between the generated sequence and the original music
    matches = sum(1 for note in sequence if note in old_music)
    
    # Compute the total length of the generated sequence and the original music
    total_length = len(sequence) + sum(len(s) for s in old_music)
    
    # Check if total_length is greater than zero to avoid zero division error
    if total_length > 0:
        # Compute the fitness score as the ratio of matching notes and chords to the total length
        fitness_score = matches / total_length
    else:
        # If total_length is zero, set the fitness score to zero
        fitness_score = 0
        
    return fitness_score

def calculate_fitness(population):
    """
    Calculate the fitness score for each individual in the population
    """
    fitness_scores = []
    for individual in population:
        # Convert the individual to a music sequence
        sequence = individual
        # Evaluate the fitness of the sequence using the evaluate_fitness() function
        fitness_score = evaluate_fitness(sequence)
        
        # Append the fitness score to the list of fitness scores
        fitness_scores.append(fitness_score)
        
    return fitness_scores

# Starting Genetic Algorithm
POPULATION_SIZE = 100
MUTATION_RATE = 0.1
GENERATIONS = 50
length = len(old_music)

# Define function and parameters to generate random music
def generate_sequence(length):
    sequence = [random.choice(old_music) for j in range(length)]
    return sequence

def generate_population(length):
    population = []
    for i in range(POPULATION_SIZE):
        sequence = generate_sequence(length)
        population.append(sequence)

    return population

# Selection function
def select_parents(population, fitness_fn):
    # Sort the population based on fitness score in descending order
    sorted_population = sorted(population, key=fitness_fn, reverse=True)
    # Select the top performers to be parents for the next generation
    top_performers = sorted_population[:int(len(sorted_population)/2)]
    return top_performers

# Crossover function
def crossover(parent1, parent2):
    # Combine the parents' sequences
    combined_sequence = parent1 + parent2
    # Randomly shuffle the combined sequence
    random.shuffle(combined_sequence)
    # Select a random crossover point
    crossover_point = random.randint(1, len(parent1)-1)
    # Perform crossover to create offspring
    offspring1 = combined_sequence[:crossover_point] + parent1[crossover_point:]
    offspring2 = combined_sequence[crossover_point:] + parent2[:crossover_point]
    return offspring1, offspring2

# Mutation function
def mutate(sequence, mutation_rate):
    # Check if sequence is empty and return as is
    if len(sequence) == 0:
        return sequence
    # Determine number of mutations based on mutation rate
    num_mutations = max(1, int(mutation_rate * len(sequence)))
    # Randomly select positions to mutate
    mutation_positions = random.sample(range(len(sequence)), num_mutations)
    # Mutate selected positions
    for position in mutation_positions:
        sequence[position] = random.choice(NOTES)
    return sequence
    
def run_genetic_algorithm(population_size, mutation_rate, generations):
    # Generate initial population
    population = generate_population(length)

    # Iterate through generations
    for i in range(generations):
        # Calculate fitness scores for each individual
        fitness_scores = calculate_fitness(population)

        # Select parents for next generation
        parents = select_parents(population, lambda x: evaluate_fitness(x))

        # Create new population by performing crossover and mutation
        offspring = []
        for j in range(population_size - len(parents)):
            parent1, parent2 = random.choices(parents, k=2)
            child1, child2 = crossover(parent1, parent2)
            if random.random() < mutation_rate:
                child1 = mutate(child1)
            if random.random() < mutation_rate:
                child2 = mutate(child2)
            offspring.append(child1)
            offspring.append(child2)

        # Combine parents and offspring to form next generation
        population = parents + offspring

    # Calculate fitness scores for final population
    fitness_scores = calculate_fitness(population)

    # Select the best individual in the final population
    best_individual = max(population, key=lambda x: evaluate_fitness(x))

    return best_individual    
    
    
    
    
    
    
    

In [ ]:
def genetic_algorithm(target_sequence, population_size, fitness_fn, mutation_rate):
    # Initialize the population with random sequences
    population = [generate_random_sequence(len(target_sequence)) for i in range(population_size)]
    generation = 0
    while True:
        # Evaluate the fitness of each sequence in the population
        fitness_scores = [fitness_fn(sequence, target_sequence) for sequence in population]
        # Check if we have found the target sequence
        if max(fitness_scores) == len(target_sequence):
            return population[fitness_scores.index(max(fitness_scores))]
        # Select the top performers to be parents for the next generation
        selected_top_performers = select_parents(population, fitness_fn)
        # Create the next generation by performing crossover and mutation on the selected top performers
        next_generation = []
        for i in range(0, len(selected_top_performers), 2):
            parent1 = selected_top_performers[i]
            parent2 = selected_top_performers[i+1]
            offspring1, offspring2 = crossover(parent1, parent2)
            next_generation.append(offspring1)
            next_generation.append(offspring2)
        # Perform mutation on the offspring
        for i in range(len(next_generation)):
            if random.random

In [ ]:
# Starting Genetic Algorithm
POPULATION_SIZE = 100
MUTATION_RATE = 0.1
GENERATIONS = 50
length = len(old_music)

# Generate initial population
population = generate_population(length)

# Iterate over the specified number of generations
for i in range(GENERATIONS):
    print("Generation ", i+1)
    
    # Evaluate the fitness of each individual in the population
    fitness_scores = calculate_fitness(population)
    
    # Select the top performers to be parents for the next generation
    parents = select_parents(population, lambda i: fitness_scores[population.index(i)])
    
    # Create the next generation through crossover and mutation
    offspring = []
    while len(offspring) < POPULATION_SIZE:
        # Select two parents from the parent pool
        parent1, parent2 = random.sample(parents, 2)
        
        # Perform crossover to create two offspring
        offspring1, offspring2 = crossover(parent1, parent2)
        
        # Mutate the offspring with a certain probability
        if random.random() < MUTATION_RATE:
            offspring1 = mutate(offspring1)
        if random.random() < MUTATION_RATE:
            offspring2 = mutate(offspring2)
        
        # Add the offspring to the new population